In [1]:
import os
from requests_html import HTML
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

In [2]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options= options) # so that browser does not get open everytime we run the code.

In [3]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.in/gp/bestsellers/electronics/ref=zg_bs_nav_0",
    "https://www.amazon.in/gp/bestsellers/garden/ref=zg_bs_nav_0"
]

In [4]:
first_url = categories[0]
driver.get(first_url)
body_el = driver.find_element_by_css_selector("body")
html_str = body_el.get_attribute("innerHTML")
# print(html_str)
html_obj = HTML(html = html_str)
# print(html_obj)

In [5]:
new_links = [x for x in html_obj.links if x.startswith("/")]
# new_links = [x for x in new_links if "product-reviews" not in x]
# print(new_links)

In [6]:
page_links = [f"https://www.amazon.com{x}" for x in new_links]
# page_links

In [7]:
def scrape_product_page(url, title_lookup="#productTitle", price_lookup="#priceblock_ourprice"):
    driver.get(url)
    print(url)
    time.sleep(1.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html = html_str)
    title = html_obj.find(title_lookup, first=True).text # first= True as find returns a list of elements
    price = html_obj.find(price_lookup, first=True).text
    return title, price

# title , price = scrape_product_page("https://www.amazon.in/Samsung-Galaxy-Space-Black-Storage/dp/B07HGN617M/ref=zg_bs_electronics_2?_encoding=UTF8&psc=1&refRID=H64JPJV0KPKTTPF3MGGP")
# print(title, price)

# for link in page_links:
#     product_price , product_title = None , None
#     try:
#         product_price , product_title = scrape_product_page(link)
#     except:
#         pass
#     if product_title != None and product_price != None:
#         print(link, product_title,product_price)
    

In [8]:
# https://www.amazon.com/Cartulina-SunWorks-de-Pacon-Negro/dp/B002LARR7Q/ref=zg_bs_toys-and-games_47/145-2143239-3431858?_encoding=UTF8&psc=1&refRID=74QVQYR37NW6EC5D1Z2X
# actual url is - https://www.amazon.com/Cartulina-SunWorks-de-Pacon-Negro/dp/B002LARR7Q
# base_url/<slug>/dp/<product_id>

In [9]:
# my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
# my_url = 'https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/'
# regex = re.compile(my_regex_pattern)
# match = regex.match(my_url)
# print(match)

In [10]:
# match['product_id']

In [11]:
# match['slug']

In [12]:
regex_options = [
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[/w-]+)",
    r"https://www.amazon.com/gp/product/(?P<product_id>[/w-]+)"
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
        return product_id

In [13]:
final_page_links = []
def clean_page_links(page_links=[]):
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url" :url, "product_id":product_id})
    return final_page_links

In [14]:
cleaned_page_links = []
cleaned_page_links = clean_page_links(page_links)
print(cleaned_page_links)

[{'url': 'https://www.amazon.com/Intex-Float-Inflatable-Lounge-Colors/dp/B000OV0X4S/ref=zg_bs_toys-and-games_9/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9', 'product_id': 'B000OV0X4S'}, {'url': 'https://www.amazon.com/Crayola-Shark-Coloring-Pages-Wonder/dp/B07PMLL5L7/ref=zg_bs_toys-and-games_15/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9', 'product_id': 'B07PMLL5L7'}, {'url': 'https://www.amazon.com/Fix-Find-flexible-colorful-Noodles/dp/B004UHED0C/ref=zg_bs_toys-and-games_28/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9', 'product_id': 'B004UHED0C'}, {'url': 'https://www.amazon.com/L-L-Surprise-Surprises-Including/dp/B07XSPCBVP/ref=zg_bs_toys-and-games_10/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9', 'product_id': 'B07XSPCBVP'}, {'url': 'https://www.amazon.com/Aqua-Multi-Purpose-Inflatable-Removable-Resistant/dp/B01N7FHWPY/ref=zg_bs_toys-and-games_1/142-2036515-3542607?_encoding=UTF8&psc=1

In [15]:
len(cleaned_page_links) != len(page_links)

True

In [16]:
len(cleaned_page_links)

50

In [17]:
len(page_links)

206

In [26]:
def perform_scrape(cleaned_items = []):
    extracted_data = []
    print(len(cleaned_items))
    for link in cleaned_items:
        url = link.get("url")
        product_id = link.get("product_id")
        product_title, product_price = None , None
        try:
            product_title, product_price = scrape_product_page(url)
         
        except:
            pass
        
        if product_title != None and product_price != None:
            product_data = {
                "url":link,
                "product_id": product_id ,
                "product_title" : product_title,
                "product_price": product_price
            }
            
            extracted_data.append(product_data)
            
    return extracted_data

In [24]:
data_extracted = perform_scrape(cleaned_items = cleaned_page_links)
print(data_extracted)

50
https://www.amazon.com/Intex-Float-Inflatable-Lounge-Colors/dp/B000OV0X4S/ref=zg_bs_toys-and-games_9/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/Crayola-Shark-Coloring-Pages-Wonder/dp/B07PMLL5L7/ref=zg_bs_toys-and-games_15/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/Fix-Find-flexible-colorful-Noodles/dp/B004UHED0C/ref=zg_bs_toys-and-games_28/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/L-L-Surprise-Surprises-Including/dp/B07XSPCBVP/ref=zg_bs_toys-and-games_10/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/Aqua-Multi-Purpose-Inflatable-Removable-Resistant/dp/B01N7FHWPY/ref=zg_bs_toys-and-games_1/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/Child-11-inch-Yoda-like-Figure-Mandalorian/dp/B0825SNHP1/ref=zg_bs_toys-and-games_26/142-2036515-3542607?_encoding=

https://www.amazon.com/Animal-Suncatcher-Refill-Horizon-Group/dp/B07KD24XBS/ref=zg_bs_toys-and-games_44/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
https://www.amazon.com/Waterproof-Stickers-Luggage-Scrapbook-Postcard/dp/B07QR3NWGK/ref=zg_bs_toys-and-games_24/142-2036515-3542607?_encoding=UTF8&psc=1&refRID=R7GXZTBFFG39JZV35DG9
[]
